## Part 1: Preprocessing

In [108]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [109]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [110]:
# Create y_df with the Attrition and Department columns

# Creating a dataframe named y_df with the Attrition and Department columns.
y_df = attrition_df[["Attrition", "Department"]].copy()
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [111]:
# Create a list of at least 10 column names to use as X data

# Creating list of 10 column names from the attrition dataframe.
column_names = ["Education", "Age", "DistanceFromHome", "JobSatisfaction"
               ,"OverTime", "StockOptionLevel", "WorkLifeBalance", "YearsAtCompany"
               ,"YearsSinceLastPromotion", "NumCompaniesWorked"]

# Create X_df using your selected columns

# Creating a datafroma from the list defined above.
X_df = pd.DataFrame(attrition_df, columns=column_names)

# Show the data types for X_df
X_df.dtypes

,0
Education,int64
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
OverTime,object
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [112]:
# Moving the Preprocessing step up prior to train_test_split call
# Convert the data and columns datatype to an integer
X_df["OverTime"] = X_df["OverTime"].astype(str).replace({"No": 0, "Yes":1})
X_df["OverTime"].value_counts()


,count
OverTime,
0,1054
1,416


In [113]:
# Verify that X_df columns are all converted to numeric by displaying the dataypes.
X_df.dtypes

,0
Education,int64
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
OverTime,int64
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [114]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)

In [115]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

###########################################################################################
# The code to convert the data to numeric can be found in 2 cells up (under pre-processing).
###########################################################################################

In [116]:
# Create a StandardScaler
# Creating a standard scaler instance.
scaler = StandardScaler()

# Fit the StandardScaler to the training data
# Fitting the scaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
# Scaling both the X_train and X_test sets.
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [117]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_encoder.fit(y_df["Department"].values.reshape(-1,1))

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department_encoded = department_encoder.transform(y_train["Department"].values.reshape(-1,1))
y_test_department_encoded = department_encoder.transform(y_test["Department"].values.reshape(-1,1))

# Creating the encoded training and test data for y_train["Department"] & y_test["Department"]
y_train_department = pd.DataFrame(y_train_department_encoded, columns=department_encoder.get_feature_names_out(["Department"]))
y_test_department = pd.DataFrame(y_test_department_encoded, columns=department_encoder.get_feature_names_out(["Department"]))


In [118]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_df["Attrition"].values.reshape(-1,1))

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition_encoded = attrition_encoder.transform(y_train["Attrition"].values.reshape(-1,1))
y_test_attrition_encoded = attrition_encoder.transform(y_test["Attrition"].values.reshape(-1,1))

# Creating the encoded training and test data for y_train["Attrition"] & y_test["Attrition"]
y_train_attrition = pd.DataFrame(y_train_attrition_encoded, columns=attrition_encoder.get_feature_names_out(["Attrition"]))
y_test_attrition = pd.DataFrame(y_test_attrition_encoded, columns=attrition_encoder.get_feature_names_out(["Attrition"]))


## Create, Compile, and Train the Model

In [119]:
# Find the number of columns in the X training data

# Find the number of columns and assign it to a variable called number_of_columns.
number_of_columns = len(X_train.columns)

# Create the input layer
input_layer = layers.Input(shape=(number_of_columns,), name="input_layer")

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation="relu", name="shared_layer1")(input_layer)
shared_layer2 = layers.Dense(128, activation="relu", name="shared_layer2")(shared_layer1)

In [120]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation="relu", name="department_hidden")(shared_layer2)

# Create the output layer
department_output = layers.Dense(3, activation="softmax", name="department_output")(department_hidden)


In [121]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation="relu", name="attrition_hidden")(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(2, activation="sigmoid", name="attrition_output")(attrition_hidden)


In [122]:
# Create the model
model = Model(inputs=input_layer, outputs=(department_output, attrition_output))

# Compile the model
model.compile(optimizer="adam",
              loss={"department_output": "categorical_crossentropy", "attrition_output": "binary_crossentropy"},
              metrics={"department_output": "accuracy", "attrition_output": "accuracy"})

# Summarize the model
model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 10)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared_layer1 (Dense)     │ (None, 64)             │            704 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared_layer2 (Dense)     │ (None, 128)            │          8,320 │ shared_layer1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_hidden (Dense) │ (None, 32)             │          4,128 │ shared_layer2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_hidden (Dense)  │ (None, 32)             │          4,128 │ shared_layer2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             99 │ department_hidden[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             66 │ attrition_hidden[0][0] │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [123]:
# Train the model
model.fit(X_train,
          {"department_output": y_train_department, "attrition_output": y_train_attrition},
          epochs=100,
          batch_size=32,
          validation_split=0.2)


Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - attrition_output_accuracy: 0.8368 - department_output_accuracy: 0.5585 - loss: 1.8416 - val_attrition_output_accuracy: 0.8281 - val_department_output_accuracy: 0.6335 - val_loss: 1.3318
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - attrition_output_accuracy: 0.8531 - department_output_accuracy: 0.6706 - loss: 1.1924 - val_attrition_output_accuracy: 0.8281 - val_department_output_accuracy: 0.6335 - val_loss: 1.3030
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - attrition_output_accuracy: 0.8430 - department_output_accuracy: 0.6563 - loss: 1.2092 - val_attrition_output_accuracy: 0.8281 - val_department_output_accuracy: 0.6335 - val_loss: 1.3093
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - attrition_output_accuracy: 0.8506 - department_output_accuracy: 0.6521 - loss: 1.1722 - val_attrition_output_accuracy: 0.8281 - val_department_output_accuracy: 0.6335 - val_loss: 1.3333
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7

In [125]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test, {"department_output": y_test_department, "attrition_output": y_test_attrition})
test_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.7730 - department_output_accuracy: 0.5436 - loss: 1.6489 


[1.540319561958313, 0.801630437374115, 0.5597826242446899]

In [126]:
# Print the accuracy for both department and attrition
print(f"Department predictions accuracy: {test_results[1]}")
print(f"Attrition predictions accuracy: {test_results[2]}")

Department predictions accuracy: 0.801630437374115
Attrition predictions accuracy: 0.5597826242446899


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
2.
3.